# Импорты

In [1]:
!pip install biopython
from Bio import SeqIO

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 8.3 MB/s eta 0:00:00


In [2]:
import re
import pandas as pd
import numpy as np

# Аннотация

In [3]:
# Считываем аннотацию
annotation_df = pd.read_csv('genomic.gff', sep='\t', comment='#', header=None, names=[
    "seqid", "source", "type", "start", "end", "score", "strand", "phase", "attributes"
])

In [11]:
# Айдишники ваших хромосом (в фаста файле есть мусор)

chr_seqid = ['NC_036159.2', 'NC_036160.2', 'NC_036161.2', 'NC_036162.2', 'NC_036163.2',
             'NC_036164.2', 'NC_036165.2', 'NC_036166.2', 'NC_036167.2', 'NC_036168.2',
             'NC_036169.2', 'NC_036170.2', 'NC_036171.2', 'NC_036172.2', 'NC_015303.1', 'NC_030892.1']

In [13]:
chr_rename = {
    'NC_036159.2' : '1',
    'NC_036160.2' : '2',
    'NC_036161.2' : '3',
    'NC_036162.2' : '4',
    'NC_036163.2' : '5',
    'NC_036164.2' : '6',
    'NC_036165.2' : '7',
    'NC_036166.2' : '8',
    'NC_036167.2' : '9',
    'NC_036168.2' : '10',
    'NC_036169.2' : '11',
    'NC_036170.2' : '12',
    'NC_036171.2' : '13',
    'NC_036172.2' : '14',
    'NC_015303.1' : 'MT',
    'NC_030892.1' : 'Pltd',
}

In [14]:
# Размеры ваших хромосом ( .sum() должен давать размер генома )

chr_df = annotation_df[(annotation_df['type']=='region') & (annotation_df['seqid'].isin(chr_seqid))]
chr_df = chr_df.reset_index(drop=True)

In [15]:
chr_df

,seqid,source,type,start,end,score,strand,phase,attributes
0,NC_036159.2,RefSeq,region,1,515659,.,+,.,ID=NC_036159.2:1..515659;Dbxref=taxon:5823;Nam...
1,NC_036160.2,RefSeq,region,1,622508,.,+,.,ID=NC_036160.2:1..622508;Dbxref=taxon:5823;Nam...
2,NC_036161.2,RefSeq,region,1,659512,.,+,.,ID=NC_036161.2:1..659512;Dbxref=taxon:5823;Nam...
3,NC_036162.2,RefSeq,region,1,712327,.,+,.,ID=NC_036162.2:1..712327;Dbxref=taxon:5823;Nam...
4,NC_036163.2,RefSeq,region,1,931174,.,+,.,ID=NC_036163.2:1..931174;Dbxref=taxon:5823;Nam...
5,NC_036164.2,RefSeq,region,1,984266,.,+,.,ID=NC_036164.2:1..984266;Dbxref=taxon:5823;Nam...
6,NC_036165.2,RefSeq,region,1,846620,.,+,.,ID=NC_036165.2:1..846620;Dbxref=taxon:5823;Nam...
7,NC_036166.2,RefSeq,region,1,1420537,.,+,.,ID=NC_036166.2:1..1420537;Dbxref=taxon:5823;Na...
8,NC_036167.2,RefSeq,region,1,1633586,.,+,.,ID=NC_036167.2:1..1633586;Dbxref=taxon:5823;Na...
9,NC_036168.2,RefSeq,region,1,1640193,.,+,.,ID=NC_036168.2:1..1640193;Dbxref=taxon:5823;Na...


In [16]:
annotation_df[(annotation_df['type']=='region') & (annotation_df['seqid'].isin(chr_seqid))]['end'].sum()

18644680

In [17]:
annotation_df[(annotation_df['type']=='region')]['end'].sum()

18777064

In [18]:
# Функция по прометке интронов, т.к. изначально их нет в genomic.gff

def add_introns(annotation_df):
    introns = []
    for gene in annotation_df[annotation_df['type'] == 'gene'].itertuples():
        gene_exons = annotation_df[(annotation_df['type'] == 'exon') &
                                   (annotation_df['seqid'] == gene.seqid) &
                                   (annotation_df['start'] >= gene.start) &
                                   (annotation_df['end'] <= gene.end)]
        gene_exons = gene_exons.sort_values(by='start')
        previous_exon_end = None
        for exon in gene_exons.itertuples():
            if previous_exon_end is not None:
                intron_start = previous_exon_end + 1
                intron_end = exon.start - 1
                if intron_start < intron_end:
                    introns.append({
                        "seqid": gene.seqid,
                        "source": "predicted",
                        "type": "intron",
                        "start": intron_start,
                        "end": intron_end,
                        "score": ".",
                        "strand": gene.strand,
                        "phase": ".",
                        "attributes": gene.attributes
                    })
            previous_exon_end = exon.end

    introns_df = pd.DataFrame(introns)
    return pd.concat([annotation_df, introns_df], ignore_index=True)

annotation_intron_df = add_introns(annotation_df)

In [19]:
# Окончательная прометка с интронами

annotation_intron_df = annotation_intron_df[annotation_intron_df['seqid'].isin(chr_seqid)].sort_values(['seqid', 'start'], ascending=[True, True])

In [20]:
# Оставляем только нужные нам типы

annotation_intron_df = annotation_intron_df[annotation_intron_df['type'].isin(['exon', 'gene', 'intron'])]

In [21]:
annotation_intron_df

,seqid,source,type,start,end,score,strand,phase,attributes
36945,NC_015303.1,RefSeq,gene,317,495,.,+,.,ID=gene-PlbeoM_r01;Dbxref=GeneID:10351799;Name...
36947,NC_015303.1,RefSeq,exon,317,495,.,+,.,ID=exon-PlbeoM_r01-1;Parent=rna-PlbeoM_r01;Dbx...
36948,NC_015303.1,RefSeq,gene,686,740,.,+,.,ID=gene-PlbeoM_r02;Dbxref=GeneID:10351785;Name...
36950,NC_015303.1,RefSeq,exon,686,740,.,+,.,ID=exon-PlbeoM_r02-1;Parent=rna-PlbeoM_r02;Dbx...
36951,NC_015303.1,RefSeq,gene,742,820,.,+,.,ID=gene-PlbeoM_r03;Dbxref=GeneID:10351796;Name...
...,...,...,...,...,...,...,...,...,...
36503,NC_036172.2,RefSeq,exon,2541560,2541946,.,+,.,ID=exon-XM_034568058.1-3;Parent=rna-XM_0345680...
36509,NC_036172.2,RefSeq,gene,2544982,2545751,.,+,.,ID=gene-PBANKA_1466241;Dbxref=GeneID:55152716;...
36511,NC_036172.2,RefSeq,exon,2544982,2545050,.,+,.,ID=exon-XM_034568059.1-1;Parent=rna-XM_0345680...
45344,NC_036172.2,predicted,intron,2545051,2545295,.,+,.,ID=gene-PBANKA_1466241;Dbxref=GeneID:55152716;...


# Инструмент для прометки попаданий в интроны, экзоны и т.д.

In [22]:
def classify_structure(structure, annotation_df):
    seqid = structure['seqid']
    start = structure['start']
    end = structure['end']
    length = end - start + 1
    midpoint = start + length // 2

    # Фильтруем аннотации по текущему seqid
    chromosome_annotations = annotation_df[annotation_df['seqid'] == seqid]

    # Фильтруем по координатам (увеличим область поиска на 1000bp для промоторов и downstream)
    filtered_annotations = chromosome_annotations[
        ((chromosome_annotations['start'] - 1000 <= end) & (chromosome_annotations['end'] + 1000 >= start))
    ]

    promoter_range = 1000
    downstream_range = 200

    for _, row in filtered_annotations.iterrows():
        if row['type'] == 'gene':
            tss = row['start'] if row['strand'] == '+' else row['end']
            tes = row['end'] if row['strand'] == '+' else row['start']

            # Проверка промотора (1000 bp upstream от TSS)
            if (row['strand'] == '+' and tss - promoter_range <= midpoint <= tss) or \
               (row['strand'] == '-' and tss <= midpoint <= tss + promoter_range):
                return "promoter"

            # Проверка downstream-области (200 bp downstream от TES)
            if (row['strand'] == '+' and tes <= midpoint <= tes + downstream_range) or \
               (row['strand'] == '-' and tes - downstream_range <= midpoint <= tes):
                return "downstream"

        # Проверка на экзон/интрон/ген
        else:
          if row['start'] <= midpoint <= row['end']:
              return row['type']

    return "intergenic"

# Квадруплексы по регулярке

## Поиск

In [48]:
pattern = r"((?:G{3,}[ATGC]{1,7}){3,}G{3,5})|((?:C{3,}[ATGC]{1,7}){3,}C{3,5})"

genome_records = list(SeqIO.parse("genome.fna", "fasta"))

# Список для хранения результатов
quad_data = []

# Поиск квадруплексов для каждой хромосомы на обоих стрендах
for record in genome_records:
    seqid = record.id
    sequence = str(record.seq)

    # Поиск квадруплексов на обоих стрендах
    for match in re.finditer(pattern, sequence, re.IGNORECASE):
        match_seq = match.group(0)

        quad_data.append({
            "seqid": seqid,
            "start": match.start() + 1,  # 1-based position
            "end": match.end(),  # 1-based position
            "sequence": match_seq
        })

# Создание DataFrame с результатами
structures_df = pd.DataFrame(quad_data)

In [49]:
structures_df = structures_df[structures_df['seqid'].isin(chr_seqid)]

In [50]:
structures_df

,seqid,start,end,sequence
0,NC_036159.2,6,736,ccctaaacccctgaaaccctgaaccctaaaccctaaaccctgaacc...
1,NC_036160.2,4,800,ccctgaaccctgaaccctaaaccctgaaccctaaaccctgaaccct...
2,NC_036160.2,186729,186754,GGGTAGCGTGGGTGGGGGGTTGAGGG
3,NC_036160.2,622167,622343,gggtttagggttcagggttcagggttcagggttcagggttcagggt...
4,NC_036160.2,622354,622506,gggttcagggtttagggttcagggttcagggtttcagggttcaggg...
5,NC_036161.2,3,75,ccctaaaccctgaaccctgaaccctgaaccctaaaccctaaaccct...
6,NC_036161.2,659312,659510,gggtttagggttcagggtttagggttcagggttcagggttcagggt...
7,NC_036162.2,6,156,ccctaaaccctgaaccctgaaccctgaaccctgaaaccctaaaccc...
8,NC_036162.2,167,232,ccctgaaccctaaaccctaaaccctaaaccctgaaccctaaaccct...
9,NC_036162.2,712203,712325,gggtttagggttcaggggtttagggttcagggtttagggttcaggg...


## Прометка позиций квадруплексов

In [51]:
# Классификация всех структур

classified_structures = []
for structure in structures_df.itertuples():
    structure_dict = structure._asdict()
    structure_type = classify_structure(structure_dict, annotation_intron_df)
    # structure_dict['type'] = structure_type
    classified_structures.append(structure_type)


In [52]:
np.unique(classified_structures, return_counts=True)

(array(['downstream', 'exon', 'intergenic', 'promoter'], dtype='<U10'),
 array([ 3,  4, 11,  5]))

In [53]:
np.unique(classified_structures, return_counts=True)[1]/(len(classified_structures))*100

array([13.04347826, 17.39130435, 47.82608696, 21.73913043])

In [54]:
structures_tobed = structures_df.rename(columns={'start': 'chromStart', 'end': 'chromEnd', 'seqid':'chr'})
structures_tobed['chr'] = structures_tobed['chr'].replace(chr_rename)
# Save the DataFrame as a BED file
bed_file = "quad.bed"
structures_tobed.to_csv(bed_file, sep='\t', header=False, index=False, columns=('chr', 'chromStart', 'chromEnd'))

# Zhunt

## Поиск

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [40]:
chunks = z=pd.read_csv("/content/drive/MyDrive/genome.fna.Z-SCORE",
              skiprows=1,
              names=["start","end","3","4","5","score","seq","8"],
              delim_whitespace=True,
              chunksize=10000)


# Список для хранения фильтрованных данных
filtered_chunks = []

# Проходим по каждому куску и фильтруем данные
for chunk in chunks:
    filtered_chunk = chunk[chunk['score'] > 4000]
    filtered_chunks.append(filtered_chunk)

# Объединяем все отфильтрованные куски в один DataFrame
filtered_df = pd.concat(filtered_chunks, ignore_index=True)

In [41]:
zhunt_df = filtered_df[filtered_df['score'] > 4000]

In [42]:
zhunt_df

,start,end,3,4,5,score,seq,8
0,84681,84697,16,20.072,44.837,4682.275,tatacatgcgcacgtg,ASASASASASASASAS
1,84683,84699,16,20.075,43.759,4662.499,tacatgcgcacgtgga,ASASASASASASASAS
2,163532,163554,22,20.114,52.633,4417.091,aattatgtacacacgtgcacac,SASASASASASASASASASASA
3,163534,163554,20,19.979,53.230,5328.222,ttatgtacacacgtgcacac,SASASASASASASASASASA
4,163536,163554,18,19.847,53.189,6420.008,atgtacacacgtgcacac,SASASASASASASASASA
...,...,...,...,...,...,...,...,...
788,18324787,18324811,24,20.131,51.982,4317.725,atatatgagcacgcacacatgcac,SASASASASASASASASASASASA
789,18324789,18324811,22,19.995,53.091,5211.324,atatgagcacgcacacatgcac,SASASASASASASASASASASA
790,18324791,18324811,20,19.866,53.529,6250.450,atgagcacgcacacatgcac,SASASASASASASASASASA
791,18324793,18324811,18,19.744,53.304,7434.581,gagcacgcacacatgcac,SASASASASASASASASA


## Прометка

In [43]:

chr_df['cumulative_end'] = chr_df['end'].cumsum()

# Создание нового DataFrame с корректными координатами и метками по хромосомам
corrected_records = []

for index, row in zhunt_df.iterrows():
    # Найти соответствующую хромосому по координатам
    for chrom_index, chrom_row in chr_df.iterrows():
        if row['start'] <= chrom_row['cumulative_end']:
            seqid = chrom_row['seqid']
            prev_cumulative_end = chr_df.iloc[chrom_index - 1]['cumulative_end'] if chrom_index > 0 else 0
            corrected_start = row['start'] - prev_cumulative_end
            corrected_end = row['end'] - prev_cumulative_end


            corrected_records.append({
                'seqid': seqid,
                'start': corrected_start,
                'end': corrected_end,
                'seq': row['seq']
            })
            break

corrected_zhunt_df = pd.DataFrame(corrected_records)


In [44]:
corrected_zhunt_df

,seqid,start,end,seq
0,NC_036159.2,84681,84697,tatacatgcgcacgtg
1,NC_036159.2,84683,84699,tacatgcgcacgtgga
2,NC_036159.2,163532,163554,aattatgtacacacgtgcacac
3,NC_036159.2,163534,163554,ttatgtacacacgtgcacac
4,NC_036159.2,163536,163554,atgtacacacgtgcacac
...,...,...,...,...
788,NC_036172.2,2266628,2266652,atatatgagcacgcacacatgcac
789,NC_036172.2,2266630,2266652,atatgagcacgcacacatgcac
790,NC_036172.2,2266632,2266652,atgagcacgcacacatgcac
791,NC_036172.2,2266634,2266652,gagcacgcacacatgcac


In [45]:
# Классификация всех структур

classified_structures_zhunt = []
for structure in corrected_zhunt_df.itertuples():
    structure_dict = structure._asdict()
    structure_type = classify_structure(structure_dict, annotation_intron_df)
    # structure_dict['type'] = structure_type
    classified_structures_zhunt.append(structure_type)


In [46]:
np.unique(classified_structures_zhunt, return_counts=True)

(array(['downstream', 'exon', 'intergenic', 'intron', 'promoter'],
       dtype='<U10'),
 array([ 78, 162, 158,  76, 319]))

In [47]:
np.unique(classified_structures_zhunt, return_counts=True)[1]/(len(classified_structures_zhunt))*100

array([ 9.83606557, 20.42875158, 19.92433796,  9.58385876, 40.22698613])

In [55]:
zhunt_tobed = corrected_zhunt_df.rename(columns={'start': 'chromStart', 'end': 'chromEnd', 'seqid':'chr'})
zhunt_tobed['chr'] = zhunt_tobed['chr'].replace(chr_rename)

bed_file = "zhunt.bed"
zhunt_tobed.to_csv(bed_file, sep='\t', header=False, index=False, columns=('chr', 'chromStart', 'chromEnd'))

# ZDNABERT

In [59]:
with open('text_predictions.txt', 'r') as file:
    lines = file.readlines()

lines = lines[1:]
# Initialize the lists to store the parsed data
seqids = []
starts = []
ends = []

# Initialize the current seqid
current_seqid = None

# Parse the lines
for line in lines:
    line = line.strip()
    if line.startswith('NC_') or line.startswith('NW_'):
        current_seqid = line
    elif line and not line.startswith('start'):
        start, end = map(int, line.split())
        seqids.append(current_seqid)
        starts.append(start)
        ends.append(end)

# Create the DataFrame
bert_df = pd.DataFrame({
    'seqid': seqids,
    'start': starts,
    'end': ends
})


In [60]:
# Классификация всех структур

classified_structures_bert = []
for structure in bert_df.itertuples():
    structure_dict = structure._asdict()
    structure_type = classify_structure(structure_dict, annotation_intron_df)
    # structure_dict['type'] = structure_type
    classified_structures_bert.append(structure_type)

In [61]:
np.unique(classified_structures_bert, return_counts=True)

(array(['exon', 'intergenic', 'intron', 'promoter'], dtype='<U10'),
 array([4, 3, 1, 3]))

In [62]:
np.unique(classified_structures_bert, return_counts=True)[1]/(len(classified_structures_bert))*100

array([36.36363636, 27.27272727,  9.09090909, 27.27272727])

In [63]:
bert_tobed = bert_df.rename(columns={'start': 'chromStart', 'end': 'chromEnd', 'seqid':'chr'})
bert_tobed['chr'] = bert_tobed['chr'].replace(chr_rename)

bed_file = "bert.bed"
bert_tobed.to_csv(bed_file, sep='\t', header=False, index=False, columns=('chr', 'chromStart', 'chromEnd'))